# Data prep and exploration

* import tables
* explore data
* join relevant info
* data prep
    * dummies
    * feature selection
* export train and test

In [2]:
import pandas as pd
import glob


In [3]:

path = r'data/' # use your path
files_list = glob.glob(path + "/*.csv")
files_list

['data/order.csv',
 'data/disp.csv',
 'data/account.csv',
 'data/card.csv',
 'data/client.csv',
 'data/loan.csv',
 'data/trans.csv',
 'data/sample.csv',
 'data/district.csv']

In [4]:
order = pd.read_csv(files_list[0], index_col=0)
disp = pd.read_csv(files_list[1], index_col=0)
account = pd.read_csv(files_list[2], index_col=0)
card = pd.read_csv(files_list[3], index_col=0)
client = pd.read_csv(files_list[4], index_col=0)
loan = pd.read_csv(files_list[5], index_col=0)
trans = pd.read_csv(files_list[6], index_col=0)
district = pd.read_csv(files_list[8], index_col=0)

/home/learner/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/learner/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
tables_list = [order, disp, account, card, client, loan, trans, district]

In [8]:
[i.shape for i in tables_list]

[(6471, 6),
 (5369, 4),
 (4500, 4),
 (892, 4),
 (5369, 4),
 (682, 7),
 (1056320, 10),
 (77, 16)]

In [9]:
[i.isna().sum() for i in tables_list]

[order_id         0
 account_id       0
 bank_to          0
 account_to       0
 amount           0
 k_symbol      1379
 dtype: int64,
 disp_id       0
 client_id     0
 account_id    0
 type          0
 dtype: int64,
 account_id     0
 district_id    0
 frequency      0
 date           0
 dtype: int64,
 card_id    0
 disp_id    0
 type       0
 issued     0
 dtype: int64,
 client_id      0
 gender         0
 birth_date     0
 district_id    0
 dtype: int64,
 loan_id       0
 account_id    0
 date          0
 amount        0
 duration      0
 payments      0
 status        0
 dtype: int64,
 trans_id           0
 account_id         0
 date               0
 type               0
 operation     183114
 amount             0
 balance            0
 k_symbol      481881
 bank          782812
 account       760931
 dtype: int64,
 district_id    0
 A2             0
 A3             0
 A4             0
 A5             0
 A6             0
 A7             0
 A8             0
 A9             0
 A10  